# Sprint ディープラーニングフレームワーク1

## 【問題1】スクラッチを振り返る
ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。


（例）


重みを初期化する必要があった
エポックのループが必要だった

それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。

### 回答
- 重みの初期化
- エポック・ミニバッチ学習（バッチ数
- レイヤー作成
- 活性化関数や出力関数
- クロスエントロピー誤差をエポックループごとに抽出
- 順伝搬での積和演算・逆伝搬で連鎖律の微分
- optimizerの用意

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf

from sklearn.preprocessing import StandardScaler

from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

## 【問題2】スクラッチとTensorFlowの対応を考える
以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」がTensorFlowではどう実装されているかを確認してください。


それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。


《サンプルコード》


＊バージョン1.5から1.14の間で動作を確認済みです。

In [3]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples    
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Instructions for updating:
Colocations handled automatically by placer.
Epoch 0, loss : 26.2877, val_loss : 60.3623, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 0.0000, val_loss : 0.4699, acc : 1.000, val_acc : 0.938
Epoch 2, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 3, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 4, loss : 0.0000, val_loss : 1.5983, acc : 1.000, val_acc : 0.938
Epoch 5, loss : 1.0690, val_loss : 6.5512, acc : 0.750, val_acc : 0.625
Epoch 6, loss : 0.0000, val_loss : 1.6757, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0000, val_loss : 0.0001, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.0000, val_loss : 0.0000, acc : 1.000, val_acc : 1.000
test_acc : 0.900


### 回答
>[問題1回答より]
1. 重みの初期化
2. エポック・ミニバッチ学習（バッチ数
3. レイヤー作成
4. 活性化関数や出力関数
5. クロスエントロピー誤差をエポックループごとに抽出
6. 順伝搬での積和演算・逆伝搬で連鎖律の微分
7. optimizerの用意

1. 重みは80~90行目で初期化。tf.Variable(tf.random_normal(前層→次層)
2. 113行目~127行目でループ。
with tf.Session() as sess:
    sess.run(init)
    この書き方が独特。
3. 90~94行目で作成
4. 90~94行目で作成
5. 99行目で損失関数の変数を作成し、ループ時は122~126行目で計算
6. 101行目でoptimizerを作成しているが、エポック・バッチ内で最適化しているコードが見当たらない。120行目のsess.run内でブラックボックス的に使われている？
7. 6同様。101行目でoptimizerを作成しているが、エポック・バッチ内で最適化しているコードが見当たらない。120行目のsess.run内でブラックボックス的に使われている？

## 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成
Irisデータセットのtrain.csvの中で、目的変数Speciesに含まれる3種類全てを分類できるモデルを作成してください。


Iris Species


2クラスの分類と3クラス以上の分類の違いを考慮してください。それがTensorFlowでどのように書き換えられるかを公式ドキュメントなどを参考に調べてください。


《ヒント》


以下の2箇所は2クラス分類特有の処理です。

In [4]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
☆重要：他クラス分類はワンホットエンコーディングが必要、n_classesはワンホットした分だけ必要。

"""
# データセットの読み込み
dataset_path ="Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
#########変更############
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
#########変更############
y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2
y = y.astype(np.int)[:, np.newaxis]

# One-Hotエンコーディングが必要
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.01   # 0.001でも試した
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3   # One-Hotエンコーディングした分
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
#########変更############
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
#########変更############
correct_pred = tf.equal(tf.argmax(Y, -1), tf.argmax(tf.nn.softmax(logits), -1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 44.9020, val_loss : 100.3276, acc : 0.667, val_acc : 0.292
Epoch 1, loss : 22.8428, val_loss : 15.2623, acc : 0.333, val_acc : 0.708
Epoch 2, loss : 5.3213, val_loss : 8.9774, acc : 0.667, val_acc : 0.708
Epoch 3, loss : 0.0000, val_loss : 0.2167, acc : 1.000, val_acc : 0.958
Epoch 4, loss : 0.0000, val_loss : 0.0899, acc : 1.000, val_acc : 0.917
Epoch 5, loss : 0.0000, val_loss : 2.1081, acc : 1.000, val_acc : 0.917
Epoch 6, loss : 0.0000, val_loss : 2.2752, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 0.7212, acc : 1.000, val_acc : 0.917
Epoch 8, loss : 0.0000, val_loss : 0.0931, acc : 1.000, val_acc : 0.958
Epoch 9, loss : 0.0000, val_loss : 1.2053, acc : 1.000, val_acc : 0.917
test_acc : 1.000


### 参考
- [@IT](https://www.atmarkit.co.jp/ait/articles/2006/08/news031_2.html)

> 前述の通り、多クラス分類問題では「Fashion-MNIST」データセットを用いる。このデータセットは、TensorFlowやtf.kerasで簡単に導入できるので、特別な準備は必要ない。Fashion-MNISTデータセットの内容は、図5に示すようなファッション商品の小さな写真であり、教師データとなるラベルには、分類カテゴリーごとに（商品カテゴリー名ではなく）0～9のクラスインデックスが指定されているので、カテゴリー変数エンコーディングの作業も不要である。

> tf.kerasのfit()メソッドにvalidation_split引数が用意されており、そこに任意の精度検証データの割合を指定できるからだ。割合として、例えば20％（＝0.2）を指定すると、訓練データが「訓練データ80％」と「精度検証データ20％（＝0.2）」に分割される（本当に便利な機能である）。



#### テスト(code)
train_labels = tf.placeholder(tf.int64, [None])
test_labels = tf.placeholder(tf.int64, [None])
train_ = tf.one_hot(train_labels, depth=3, dtype=tf.float32)
test_ = tf.one_hot(test_labels, depth=3, dtype=tf.float32)

x = tf.placeholder(tf.float32, [None, 4])
W = tf.Variable(tf.zeros([4, 3]))
b = tf.Variable(tf.zeros([3]))

y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 3])

y_

## 【問題4】House Pricesのモデルを作成
回帰問題のデータセットであるHouse Pricesを使用したモデルを作成してください。


House Prices: Advanced Regression Techniques


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使ってください。説明変数はさらに増やしても構いません。


分類問題と回帰問題の違いを考慮してください。

### 参考
- [Qlita](https://qiita.com/MahoTakara/items/0d7284774c2adf1f05ec)
- [blog記事](https://qiita.com/MahoTakara/items/0d7284774c2adf1f05ec)
- [英語記事](http://aqibsaeed.github.io/2016-07-07-TensorflowLR//)

In [39]:
"""
TensorFlowで実装したニューラルネットワークを使いHouse Priceの回帰問題
"""
# データセットの読み込み
dataset_path ="train.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
# データフレームから条件抽出
#########変更############
y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]
y = np.array(y)
X = np.array(X)
y = y.astype(np.int)[:, np.newaxis]

# print("notscaler X", X)

#正規化
ss = StandardScaler()
ss.fit(X)
X_scaler = ss.transform(X)

# yを対数変換
y_log = np.log(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X_scaler, y_log, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# print("X_train", X_train[:10], X_train.shape)
# print("X_val", X_val[:10], X_val.shape)

# ハイパーパラメータの設定
learning_rate = 0.001
batch_size = 50
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1

# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(0.01*tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(0.01*tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(0.01*tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(0.01*tf.random_normal([n_hidden1])),
        'b2': tf.Variable(0.01*tf.random_normal([n_hidden2])),
        'b3': tf.Variable(0.01*tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

#ネットワーク構造の読み込み                               
logits = example_net(X)

# 目的関数
#########変更############
loss_op = tf.reduce_mean(tf.square(Y - logits))
# 最適化手法
#########変更############
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
#########変更############
# correct_pred = example_net(X)
# 指標値計算
#########変更############
MSE = tf.losses.mean_squared_error(Y, logits)
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_mse = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss = sess.run(loss_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            mse = sess.run(MSE, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_mse += mse
        total_loss /= total_batch
        total_mse /= total_batch
        val_loss = sess.run(loss_op, feed_dict={X: X_val, Y: y_val})
        val_mse = sess.run(MSE, feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, mse : {:.3f}, val_mse : {:.3f}".format(epoch, loss, val_loss, mse, val_mse))
        test_mse = sess.run(MSE, feed_dict={X: X_test, Y: y_test})
print("test_mse : {:.3f}".format(test_mse))

/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/adachi-yuya/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 0, loss : 132.8222, val_loss : 133.8563, mse : 132.822, val_mse : 133.856
Epoch 1, loss : 122.9074, val_loss : 123.9034, mse : 122.907, val_mse : 123.903
Epoch 2, loss : 113.5693, val_loss : 114.5280, mse : 113.569, val_mse : 114.528
Epoch 3, loss : 104.6044, val_loss : 105.5258, mse : 104.604, val_mse : 105.526
Epoch 4, loss : 95.6936, val_loss : 96.5764, mse : 95.694, val_mse : 96.576
Epoch 5, loss : 86.3345, val_loss : 87.1750, mse : 86.335, val_mse : 87.175
Epoch 6, loss : 75.8011, val_loss : 76.5913, mse : 75.801, val_mse : 76.591
Epoch 7, loss : 63.2686, val_loss : 63.9946, mse : 63.269, val_mse : 63.995
Epoch 8, loss : 48.3642, val_loss : 49.0046, mse : 48.364, val_mse : 49.005
Epoch 9, loss : 32.1718, val_loss : 32.7023, mse : 32.172, val_mse : 32.702
test_mse : 32.875


## 【問題5】MNISTのモデルを作成
ニューラルネットワークのスクラッチで使用したMNISTを分類するモデルを作成してください。


3クラス以上の分類という点ではひとつ前のIrisと同様です。入力が画像であるという点で異なります。


スクラッチで実装したモデルの再現を目指してください。


### 参考
- [Tensolflowページ　fashion_mnist](https://www.tensorflow.org/tutorials/keras/classification?hl=jam)

In [41]:
"""
TensorFlowで実装したニューラルネットワークを使いMnistデータセットの他クラス分類
☆重要：他クラス分類はワンホットエンコーディングが必要、n_classesはワンホットした分だけ必要。

"""
# データセットの読み込みから前処理まで
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# One-Hotエンコーディングが必要
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# さらにtrainとvalに分割
X_train, X_val, y_train_one_hot, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)
    
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 100
num_epochs = 20
n_hidden1 = 400
n_hidden2 = 200
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10   # One-Hotエンコーディングした分
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train_one_hot, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output

# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
#########変更############
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
#########変更############
correct_pred = tf.equal(tf.argmax(Y, -1), tf.argmax(tf.nn.softmax(logits), -1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test_one_hot})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 11.7788, val_loss : 11.8515, acc : 0.930, val_acc : 0.927
Epoch 1, loss : 2.1178, val_loss : 7.9931, acc : 0.980, val_acc : 0.935
Epoch 2, loss : 2.9699, val_loss : 5.8170, acc : 0.960, val_acc : 0.943
Epoch 3, loss : 0.3035, val_loss : 5.1044, acc : 0.970, val_acc : 0.949
Epoch 4, loss : 0.2138, val_loss : 4.8975, acc : 0.990, val_acc : 0.951
Epoch 5, loss : 0.7413, val_loss : 4.1793, acc : 0.980, val_acc : 0.951
Epoch 6, loss : 0.0000, val_loss : 4.1595, acc : 1.000, val_acc : 0.955
Epoch 7, loss : 0.1682, val_loss : 3.3513, acc : 0.990, val_acc : 0.957
Epoch 8, loss : 0.2423, val_loss : 3.9461, acc : 0.990, val_acc : 0.956
Epoch 9, loss : 0.0001, val_loss : 2.7977, acc : 1.000, val_acc : 0.955
Epoch 10, loss : 0.2804, val_loss : 3.0063, acc : 0.980, val_acc : 0.956
Epoch 11, loss : 0.2374, val_loss : 2.2707, acc : 0.990, val_acc : 0.961
Epoch 12, loss : 0.7166, val_loss : 2.4544, acc : 0.970, val_acc : 0.959
Epoch 13, loss : 0.3174, val_loss : 2.4505, acc : 0.980, va

### 問題3の流用で動いた